In [7]:
# отключаем предупреждения 
import warnings
warnings.simplefilter('ignore')

import pandas as pd
import numpy as np

In [2]:
gl = pd.read_csv('./game_logs.csv')
gl.head()

,date,number_of_game,day_of_week,v_name,v_league,v_game_number,h_name,h_league,h_game_number,v_score,...,h_player_7_name,h_player_7_def_pos,h_player_8_id,h_player_8_name,h_player_8_def_pos,h_player_9_id,h_player_9_name,h_player_9_def_pos,additional_info,acquisition_info
0,18710504,0,Thu,CL1,na,1,FW1,na,1,0,...,Ed Mincher,7.0,mcdej101,James McDermott,8.0,kellb105,Bill Kelly,9.0,NaN,Y
1,18710505,0,Fri,BS1,na,1,WS3,na,1,20,...,Asa Brainard,1.0,burrh101,Henry Burroughs,9.0,berth101,Henry Berthrong,8.0,HTBF,Y
2,18710506,0,Sat,CL1,na,2,RC1,na,1,12,...,Pony Sager,6.0,birdg101,George Bird,7.0,stirg101,Gat Stires,9.0,NaN,Y
3,18710508,0,Mon,CL1,na,3,CH1,na,1,12,...,Ed Duffy,6.0,pinke101,Ed Pinkham,5.0,zettg101,George Zettlein,1.0,NaN,Y
4,18710509,0,Tue,BS1,na,2,TRO,na,1,9,...,Steve Bellan,5.0,pikel101,Lip Pike,3.0,cravb101,Bill Craver,6.0,HTBF,Y


In [3]:
# с помощью метода .info() получим основные сведения  о датафрейме, в том числе и об объеме занимаемой им памяти.
# чтобы получить точный размер в памяти необходимо в параметре memory_usage установить значение 'deep'
gl.info(memory_usage = 'deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 171907 entries, 0 to 171906
Columns: 161 entries, date to acquisition_info
dtypes: float64(77), int64(6), object(78)
memory usage: 859.4 MB


In [6]:
# посмотрим, сколько памяти в среднем используют столбцы определенного типа?
for dtype in ['float', 'int', 'object']:
    selected_dtype = gl.select_dtypes(include=[dtype])
    mean_usage_b = selected_dtype.memory_usage(deep=True).mean()
    mean_usage_mb = mean_usage_b / 1024 ** 2
    print ('Использование памяти в среднем для {} столбцов: {:03.2f} MB'. format(dtype, mean_usage_mb))

Использование памяти в среднем для float столбцов: 1.29 MB
Использование памяти в среднем для int столбцов: 1.12 MB
Использование памяти в среднем для object столбцов: 9.50 MB


__ПОДТИПЫ__ 

In [8]:
# посмотрим минимальное и максимальное значения для каждого целочисленного типа
int_types = ['uint8', 'int8', 'int16', 'int32', 'int64']
for it in int_types:
    print(np.iinfo(it))

Machine parameters for uint8
---------------------------------------------------------------
min = 0
max = 255
---------------------------------------------------------------

Machine parameters for int8
---------------------------------------------------------------
min = -128
max = 127
---------------------------------------------------------------

Machine parameters for int16
---------------------------------------------------------------
min = -32768
max = 32767
---------------------------------------------------------------

Machine parameters for int32
---------------------------------------------------------------
min = -2147483648
max = 2147483647
---------------------------------------------------------------

Machine parameters for int64
---------------------------------------------------------------
min = -9223372036854775808
max = 9223372036854775807
---------------------------------------------------------------



__ОПТИМИЗАЦИЯ ЧИСЛОВЫХ СТОЛБЦОВ С ПОМОЩЬЮ ПОНИЖАЮЩЕГО ПРЕОБРАЗОВАНИЯ__

In [ ]:
# используем функцию pd.to_numeric(), чтобы выполнить понижающее преобразования наших числовых типов. 
# воспользуемся DataFrame.select_dtypes для выбора только целочисленных столбцов

In [9]:
# напишем функцию для подсчета памяти
def mem_usage(pandas_obj):
    if isinstance(pandas_obj, pd.DataFrame):
        usage_b = pandas_obj.memory_usage(deep = True).sum()
    else:                                                    # если это не DataFrame а Series
        usage_b = pandas_obj.memory_usage(deep = True)
    usage_mb = usage_b / 1024 ** 2                           # преобразуем байты в мегабайты
    return'{:03.2f} MB'.format(usage_mb)

In [10]:
# выполняем понижающее преобразование для столбцов типа int
gl_int = gl.select_dtypes(include = ['int'])
converted_int = gl_int.apply(pd.to_numeric, downcast = 'unsigned')

print(mem_usage(gl_int))
print(mem_usage(converted_int))

compare_ints = pd.concat([gl_int.dtypes, converted_int.dtypes], axis = 1)
compare_ints.columns = ['before', 'after']
compare_ints.apply(pd.Series.value_counts)

7.87 MB
1.48 MB


,before,after
uint8,NaN,5.0
uint32,NaN,1.0
int64,6.0,NaN


In [13]:
# выполняем понижающее преобразование для столбцов типа float
gl_float = gl.select_dtypes(include = ['float'])
converted_float = gl_float.apply(pd.to_numeric, downcast = 'float')

print(mem_usage(gl_float))
print(mem_usage(converted_float))

compare_ints = pd.concat([gl_float.dtypes, converted_float.dtypes], axis = 1)
compare_ints.columns = ['before', 'after']
compare_ints.apply(pd.Series.value_counts)

100.99 MB
50.49 MB


,before,after
float32,NaN,77.0
float64,77.0,NaN


In [15]:
# создаем копию исходного датафрейма
optimized_gl = gl.copy()

# заменяем исходные числовые столбцы оптимизированными
optimized_gl[converted_int.columns] = converted_int
optimized_gl[converted_float.columns] = converted_float

# смотрим использование памяти
print(mem_usage(gl))
print(mem_usage(optimized_gl))

859.43 MB
802.54 MB


__ОПТИМИЗАЦИЯ ТИПОВ OBJECT С ПОМОЩЬЮ ТИПА CATEGORY__

In [16]:
# смотрим количество уникальных значений по каждому столбцу типа object
gl_obj = gl.select_dtypes(include = ['object']).copy()
gl_obj.describe()

,day_of_week,v_name,v_league,h_name,h_league,day_night,completion,forefeit,protest,park_id,...,h_player_6_id,h_player_6_name,h_player_7_id,h_player_7_name,h_player_8_id,h_player_8_name,h_player_9_id,h_player_9_name,additional_info,acquisition_info
count,171907,171907,171907,171907,171907,140150,116,145,180,171907,...,140838,140838,140838,140838,140838,140838,140838,140838,1456,140841
unique,7,148,7,148,7,2,116,3,5,245,...,4774,4720,5253,5197,4760,4710,5193,5142,332,1
top,Sat,CHN,NL,CHN,NL,D,"19200904,,0,6,36",H,V,STL07,...,grimc101,Charlie Grimm,grimc101,Charlie Grimm,lopea102,Al Lopez,spahw101,Warren Spahn,HTBF,Y
freq,28891,8870,88866,9024,88867,82724,1,69,90,7022,...,427,427,491,491,676,676,339,339,1112,140841


In [17]:
# запишем столбец day_of_week как dow
dow = gl_obj.day_of_week
print(dow.head())

# присваиваем столбцу dow тип category 
dow_cat = dow.astype('category')
print(dow_cat.head())

0    Thu
1    Fri
2    Sat
3    Mon
4    Tue
Name: day_of_week, dtype: object
0    Thu
1    Fri
2    Sat
3    Mon
4    Tue
Name: day_of_week, dtype: category
Categories (7, object): ['Fri', 'Mon', 'Sat', 'Sun', 'Thu', 'Tue', 'Wed']


In [18]:
# воспользуемся атрибутом Series.cat.codes. Он возвращает целочисленные значения использующиеся типом category для 
# представления каждого значения
dow_cat.head().cat.codes

0    4
1    0
2    2
3    1
4    5
dtype: int8

In [19]:
# смотрим сколько памяти использует столбец dow до и после преобразования в тип category 
print(mem_usage(dow))
print(mem_usage(dow_cat))

9.84 MB
0.16 MB


In [21]:
# напишем цикл который переберет все столбцы object, проверит его на соответствие заданному порогу (количество уникальных
# значений должно быть меньше 50% от общего количества значений) и если столбец удовлетворяет порогу преобразовывает его
# в тип category 
for col in gl_obj.columns:
    num_unique_values = len(gl_obj[col].unique())
    num_total_values = len(gl_obj[col])
    if num_unique_values / num_total_values < 0.5:
        converted_obj.loc[:, col] = gl_obj[col].astype('category')
    else:
        converted_obj.loc[:, col] = gl_obj[col]

NameError: name 'converted_obj' is not defined